In [1]:
import numpy as np, pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import nltk
from nltk.util import ngrams
random.seed(134)

PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32

In [2]:
train_tmp = open("hw2_data/snli_train.tsv").read().split('\n')
train_data = [row.split('\t') for row in train_tmp][1:-1]
val_tmp = open("hw2_data/snli_val.tsv").read().split('\n')
val_data = [row.split('\t') for row in val_tmp][1:-1]

In [3]:
train_s1 = [row[0] for row in train_data]
train_s2 = [row[1] for row in train_data]
val_s1 = [row[0] for row in val_data]
val_s2 = [row[1] for row in val_data]


In [4]:
train_label = [row[2] for row in train_data]
val_label = [row[2] for row in val_data]

In [5]:
#tokenize
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
#         print(type(sample))
        tokens = nltk.word_tokenize(sample.lower())
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

# val set tokens
print ("Tokenizing val data")
val_s1_tokens, _ = tokenize_dataset(val_s1)
pkl.dump(val_s1_tokens, open("val_s1_tokens.p", "wb"))
val_s2_tokens, _ = tokenize_dataset(val_s2)
pkl.dump(val_s2_tokens, open("val_s2_tokens.p", "wb"))

# train set tokens
print ("Tokenizing train data")
train_s1_tokens, all_train_s1_tokens = tokenize_dataset(train_s1)
train_s2_tokens, all_train_s2_tokens = tokenize_dataset(train_s2)

pkl.dump(train_s1_tokens, open("train_s1_tokens.p", "wb"))
pkl.dump(train_s2_tokens, open("train_s2_tokens.p", "wb"))


Tokenizing val data
Tokenizing train data


In [6]:
all_train_s1_tokens.extend(all_train_s2_tokens)

In [9]:
with open('wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft = np.zeros((2+600000, 300))
    words_ft = {}
    idx2words_ft = {}
    ordered_words_ft = []
    for i, line in enumerate(f):
        if i  >= 600000:
            break
        s = line.split()
        loaded_embeddings_ft[i+2, :] = np.asarray(s[1:])
        words_ft[s[0]] = i+2
        idx2words_ft[i+2] = s[0]
        ordered_words_ft.append(s[0])

In [10]:
idx2words_ft[0] = '<pad>'
idx2words_ft[1] = '<unk>'
words_ft['<pad>'] = 0
words_ft['<unk>'] = 1


In [11]:
# Lets check the dictionary by loading random token from it

random_token_id = random.randint(0, len(idx2words_ft)-1)
random_token = idx2words_ft[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, idx2words_ft[random_token_id]))
print ("Token {}; token id {}".format(random_token, words_ft[random_token]))

Token id 507635 ; token scouter
Token scouter; token id 507635


In [12]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [words_ft[token] if token in words_ft else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_s1_indices = token2index_dataset(train_s1_tokens)
val_s1_indices = token2index_dataset(val_s1_tokens)
train_s2_indices = token2index_dataset(train_s2_tokens)
val_s2_indices = token2index_dataset(val_s2_tokens)

# double checking
print ("Train dataset size is {}".format(len(train_s1_indices)))
print ("Train dataset size is {}".format(len(train_s2_indices)))
print ("Val dataset size is {}".format(len(val_s1_indices)))
print ("Val dataset size is {}".format(len(val_s2_indices)))

Train dataset size is 100000
Train dataset size is 100000
Val dataset size is 1000
Val dataset size is 1000


In [18]:
MAX_SENTENCE_LENGTH = 40
class SNLIDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, s1_data, s2_data, target_list):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.s1_data = s1_data
        self.s2_data = s2_data
        self.target_list = target_list
        assert (len(self.s1_data) == len(self.target_list))
        assert (len(self.s2_data) == len(self.target_list))
#         self.words_ft = words_ft

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        item = dict()
        
        item['s1_word_idx'] = self.s1_data[key][:MAX_SENTENCE_LENGTH]
        item['s2_word_idx'] = self.s2_data[key][:MAX_SENTENCE_LENGTH]
        item['label'] = self.target_list[key]
        return item


In [20]:
def vocab_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    s1_list, s2_list = [],[]
    label_list = []
#     length_list = []
#     print(batch)
    for datum in batch:
        label_list.append(datum['label'])
#         length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum['s1_word_idx']),
                                pad_width=((0,MAX_SENTENCE_LENGTH-len(datum['s1_word_idx']))),
                                mode="constant", constant_values=0)
        s1_list.append(list(padded_vec))
    for datum in batch:
        padded_vec = np.pad(np.array(datum['s2_word_idx']),
                                pad_width=((0,MAX_SENTENCE_LENGTH-len(datum['s2_word_idx']))),
                                mode="constant", constant_values=0)
        s2_list.append(list(padded_vec))
        
    label_list = [{'entailment':0,'contradiction':1,'neutral':2}[k] for k in label_list]
#     print(label_list)
    return [torch.from_numpy(np.array(s1_list)),torch.from_numpy(np.array(s2_list)), torch.LongTensor(label_list)]


In [21]:
train_loader = torch.utils.data.DataLoader(
    SNLIDataset(train_s1_indices, train_s2_indices, train_label),batch_size=BATCH_SIZE,shuffle=True,collate_fn=vocab_collate_func)

val_loader = torch.utils.data.DataLoader(
    SNLIDataset(val_s1_indices, val_s2_indices, val_label),batch_size=BATCH_SIZE,shuffle=True,collate_fn=vocab_collate_func)

In [22]:
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for sample in loader:
        try:
            outputs = F.softmax(model(sample[0], sample[1]), dim=1)
            predicted = outputs.max(1, keepdim=True)[1].view(-1)
            total += len(predicted)
            truths = sample[2]
            correct += predicted.eq(truths.view_as(predicted)).sum().item()
        except:
            total += 0
            correct += 0
    return (100 * correct / total)

In [35]:
class CNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers, num_classes, k=3, p=1):
        super().__init__()
        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights_matrix))
        num_ebd, emb_size = weights_matrix.shape
        
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=k, padding=p)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=k, padding=p)
#         self.cnn3 = nn.Conv1d(emb_size, hidden_size, kernel_size=5, padding=2)
#         self.cnn4 = nn.Conv1d(hidden_size, hidden_size, kernel_size=5, padding=2)
        self.tanh = nn.Tanh()
        self.relu=nn.ReLU()  
        self.drop = nn.Dropout(p = 0.2)
#         self.maxpool = nn.MaxPool1d(MAX_SENTENCE_LENGTH)
#         self.linear = nn.Linear(hidden_size*2, num_classes)
        self.linear1 = nn.Linear(hidden_size*2, hidden_size*2)
        self.linear2 = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, s1, s2):
        # reset hidden state

#         batch_size = x.size()
        batch_size1, seq_len1 = s1.size()
        batch_size2, seq_len2 = s2.size()
#         print(len(s1)）

        # get embedding of characters
        s1_embed = self.embedding(s1)
        s2_embed = self.embedding(s2)
        
        hidden1 = self.conv1(s1_embed.transpose(1,2)).transpose(1,2)
        hidden2 = self.conv1(s2_embed.transpose(1,2)).transpose(1,2)
        
        hidden1 = F.relu(hidden1.contiguous().view(-1, hidden1.size(-1))).view(batch_size1, hidden1.size(1), hidden1.size(-1))
        hidden2 = F.relu(hidden2.contiguous().view(-1, hidden2.size(-1))).view(batch_size2, hidden2.size(1), hidden2.size(-1))
        
        hidden1 = self.conv2(hidden1.transpose(1,2)).transpose(1,2)
        hidden2 = self.conv2(hidden2.transpose(1,2)).transpose(1,2)
        
        hidden1 = F.relu(hidden1.contiguous().view(-1, hidden1.size(-1))).view(batch_size1, hidden1.size(1), hidden1.size(-1))
        hidden2 = F.relu(hidden2.contiguous().view(-1, hidden2.size(-1))).view(batch_size2, hidden2.size(1), hidden2.size(-1))
        
        hidden1 = torch.sum(hidden1, dim=1)
        hidden2 = torch.sum(hidden2, dim=1)
        
        cnn_out = torch.cat([hidden1, hidden2], 1)
        logits = self.relu(self.linear1(cnn_out))
        logits = self.drop(logits)
        logits = self.linear2(logits)
            
        return logits

In [34]:
model = 2(weights_matrix=loaded_embeddings_ft, hidden_size=50, num_layers=2, num_classes=3)

learning_rate = 1e-4
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, sample in enumerate(train_loader):
#         print(len(sample['label']))
        model.train()
        optimizer.zero_grad()
        # Forward pass
        output = model(sample[0], sample[1])
        label = sample[2]
        loss = criterion(output, label)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 10 == 0:
            print(loss.item())
#             validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))

1.0648167133331299
Epoch: [1/10], Step: [11/3125], Validation Acc: 35.3
1.147667646408081
Epoch: [1/10], Step: [21/3125], Validation Acc: 33.8
1.1114165782928467
Epoch: [1/10], Step: [31/3125], Validation Acc: 37.5
1.062389612197876
Epoch: [1/10], Step: [41/3125], Validation Acc: 34.5
1.114782452583313
Epoch: [1/10], Step: [51/3125], Validation Acc: 38.0
1.1038511991500854
Epoch: [1/10], Step: [61/3125], Validation Acc: 38.7
1.1240640878677368
Epoch: [1/10], Step: [71/3125], Validation Acc: 33.6
1.1082526445388794
Epoch: [1/10], Step: [81/3125], Validation Acc: 39.46280991735537
1.1282885074615479
Epoch: [1/10], Step: [91/3125], Validation Acc: 37.6
1.1019346714019775
Epoch: [1/10], Step: [101/3125], Validation Acc: 36.0
1.1177693605422974
Epoch: [1/10], Step: [111/3125], Validation Acc: 38.8
1.0576212406158447
Epoch: [1/10], Step: [121/3125], Validation Acc: 39.4
1.112605094909668
Epoch: [1/10], Step: [131/3125], Validation Acc: 38.7
1.0809595584869385
Epoch: [1/10], Step: [141/3125],

KeyboardInterrupt: 

ma2: 2/2
arima: 0/2
acf: 2/2
hmm: 2/2
hmm&alp: 1/2
pred: 2/3
hmm_draw: 1/1
arrow: 3/3
arrow: 3/3
A: 0/2
alp&v: 1/2
A^m: 1/2
ploy: 4/4
parti: 2/2
